In [1]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import random
import torch
import torch.nn as nn
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import EvalCallback
import matplotlib.pyplot as plt

2024-11-10 15:33:07.171882: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-10 15:33:07.181320: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731241987.192821  129905 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731241987.196255  129905 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-10 15:33:07.207980: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
class Env(gym.Env):
    def __init__(self, n_candidates=100):
        super(Env, self).__init__()
        # Действия: 0 - отказ, 1 - принять
        self.action_space = spaces.Discrete(2)
        # Наблюдения: [позиция, лучше_предыдущих, относительный_ранг]
        self.observation_space = spaces.Box(
            low=np.array([0, 0, 0]), 
            high=np.array([1, 1, 1]), 
            dtype=np.float32
        )
        
        self.n_candidates = n_candidates
        self.optimal_stopping = int(self.n_candidates / np.e)
        self.reset()

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        
        # Генерация новых случайных кандидатов
        self.candidates = np.random.permutation(self.n_candidates)
        self.best_so_far = float('-inf')
        self.current_pos = 0
        self.best_candidate = np.max(self.candidates)
        self.seen_candidates = []
        
        return self._get_observation(), {}

    def _get_observation(self):
        if self.current_pos >= self.n_candidates:
            return np.array([1.0, 0.0, 0.0], dtype=np.float32)
            
        normalized_pos = self.current_pos / self.n_candidates
        is_better = float(self.candidates[self.current_pos] > self.best_so_far)
        relative_rank = 0.0
        
        if self.current_pos > 0:
            current_rank = sum(1 for x in self.seen_candidates if x > self.candidates[self.current_pos])
            relative_rank = current_rank / len(self.seen_candidates)
        
        return np.array([normalized_pos, is_better, relative_rank], dtype=np.float32)

    def step(self, action):
        if self.current_pos >= self.n_candidates:
            return self._get_observation(), -1.0, True, False, {}

        current_candidate = self.candidates[self.current_pos]
        terminated = False
        truncated = False
        reward = 0.0

        if action == 1:  # Принять кандидата
            if current_candidate == self.best_candidate:
                reward = 1.0
            else:
                rank_diff = (self.best_candidate - current_candidate) / self.n_candidates
                reward = -1.0 - rank_diff # Штраф за неоптимальный выбор
            terminated = True
        else: # Отказать
            self.seen_candidates.append(current_candidate)
            self.best_so_far = max(self.best_so_far, current_candidate)
            self.current_pos += 1
            
            reward = -0.01
            
            if self.current_pos >= self.n_candidates:
                reward = -1.0 # Малый штраф за отказ
                terminated = True
            elif self.current_pos < self.optimal_stopping:
                reward += 0.01  # Бонус за исследование до оптимальной 

        return self._get_observation(), reward, terminated, truncated, {}

    def render(self, mode='human'):
        pass


In [4]:
def create_model(env):
    return PPO(
        "MlpPolicy",
        env,
        learning_rate=1e-4,
        n_steps=1024,
        batch_size=128,
        n_epochs=20,
        gamma=0.99,
        gae_lambda=0.95,
        clip_range=0.2,
        ent_coef=0.01,  
        policy_kwargs=dict(
            net_arch=dict(
                pi=[128, 128],
                vf=[128, 128]
            )
        ),
        verbose=0
    )


In [5]:
def eval_probability(env, model, n_episodes=50, log=False):
    successes = 0
    total_reward = 0
    
    for episode in range(n_episodes):
        obs, _ = env.reset()
        done = False
        episode_reward = 0
        
        while not done:
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, terminated, _, _ = env.step(action)
            episode_reward += reward
            done = terminated
            
            if done and reward > 0: # Выбран лучший кандидат
                successes += 1
        
        total_reward += episode_reward
        
        if log and episode % 10 == 0:
            print(f"Эпизод {episode}, Успешность: {successes/(episode+1):.2f}")
    return successes / n_episodes

In [6]:
def train_model(env, model, total_timesteps=500000, eval_interval=10000, eval_episodes=100):
    evaluation_results = []
    running_avg = []
    window_size = 5  
    
    timesteps_so_far = 0
    while timesteps_so_far < total_timesteps:
        model.learn(eval_interval)
        timesteps_so_far += eval_interval
        
        success_rate = eval_probability(env.envs[0], model, eval_episodes)
        evaluation_results.append(success_rate)
        
        running_avg.append(success_rate)
        if len(running_avg) > window_size:
            running_avg.pop(0)
        
        print(f"Шаг {timesteps_so_far}/{total_timesteps}, "
              f"Успешность: {success_rate:.3f}")

    return evaluation_results

In [7]:
n_candidates = 100
env = Env(n_candidates)
env = DummyVecEnv([lambda: env])
    
model = create_model(env)
training_results = train_model(env, model, total_timesteps=500000, eval_interval=10000)

Шаг 10000/500000, Успешность: 0.010
Шаг 20000/500000, Успешность: 0.170
Шаг 30000/500000, Успешность: 0.300
Шаг 40000/500000, Успешность: 0.320
Шаг 50000/500000, Успешность: 0.370
Шаг 60000/500000, Успешность: 0.350
Шаг 70000/500000, Успешность: 0.500
Шаг 80000/500000, Успешность: 0.370
Шаг 90000/500000, Успешность: 0.400
Шаг 100000/500000, Успешность: 0.380
Шаг 110000/500000, Успешность: 0.300
Шаг 120000/500000, Успешность: 0.330
Шаг 130000/500000, Успешность: 0.330
Шаг 140000/500000, Успешность: 0.360
Шаг 150000/500000, Успешность: 0.300
Шаг 160000/500000, Успешность: 0.300
Шаг 170000/500000, Успешность: 0.400
Шаг 180000/500000, Успешность: 0.390
Шаг 190000/500000, Успешность: 0.310
Шаг 200000/500000, Успешность: 0.420
Шаг 210000/500000, Успешность: 0.400
Шаг 220000/500000, Успешность: 0.360
Шаг 230000/500000, Успешность: 0.360
Шаг 240000/500000, Успешность: 0.360
Шаг 250000/500000, Успешность: 0.380
Шаг 260000/500000, Успешность: 0.470
Шаг 270000/500000, Успешность: 0.380
Шаг 280000

In [8]:
final_success_rate = eval_probability(env.envs[0], model, n_episodes=1000, log=True)
print(f"\nИтоговая успешность модели RL: {final_success_rate:.3f}")
print(f"Теоретическая оптимальная успешность: 0.37")

Эпизод 0, Успешность: 1.00
Эпизод 10, Успешность: 0.36
Эпизод 20, Успешность: 0.38
Эпизод 30, Успешность: 0.45
Эпизод 40, Успешность: 0.39
Эпизод 50, Успешность: 0.39
Эпизод 60, Успешность: 0.39
Эпизод 70, Успешность: 0.38
Эпизод 80, Успешность: 0.37
Эпизод 90, Успешность: 0.38
Эпизод 100, Успешность: 0.39
Эпизод 110, Успешность: 0.40
Эпизод 120, Успешность: 0.40
Эпизод 130, Успешность: 0.38
Эпизод 140, Успешность: 0.38
Эпизод 150, Успешность: 0.37
Эпизод 160, Успешность: 0.37
Эпизод 170, Успешность: 0.36
Эпизод 180, Успешность: 0.36
Эпизод 190, Успешность: 0.36
Эпизод 200, Успешность: 0.35
Эпизод 210, Успешность: 0.35
Эпизод 220, Успешность: 0.36
Эпизод 230, Успешность: 0.35
Эпизод 240, Успешность: 0.35
Эпизод 250, Успешность: 0.36
Эпизод 260, Успешность: 0.36
Эпизод 270, Успешность: 0.37
Эпизод 280, Успешность: 0.36
Эпизод 290, Успешность: 0.36
Эпизод 300, Успешность: 0.36
Эпизод 310, Успешность: 0.36
Эпизод 320, Успешность: 0.36
Эпизод 330, Успешность: 0.36
Эпизод 340, Успешность: 0